## LLm Powerd ChatBot
#### This Chatbot will able to have conversation and remember previous intraction



###### Note that this chatbot we build will only use the language model to have a conversation. There are several other related concept that you may be looking for 

###### - Conversational RAG : Enable a chatbot experience over an external source of data

###### - Agent : Build a chatbot that can take action 

In [1]:
### Open AI Key and open source model configuration
import os 
from  dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## for langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT_NAME"] = "LangchainFramework"



In [2]:
import os
from langchain_groq import ChatGroq

ChatGroq.api_key = os.getenv("GROQ_API_KEY")

model = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
model

c:\Users\sarvesh\Desktop\LangchainFramework\venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
c:\Users\sarvesh\Desktop\LangchainFramework\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000019838E6C590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019838E6D010>, model_name='llama-3.3-70b-versatile', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage    
response = model.generate([ [HumanMessage(content="My name is sarvesh and I am a data scientist")]])
response

LLMResult(generations=[[ChatGeneration(text="Nice to meet you, Sarvesh! As a data scientist, you must be working with data to extract insights and knowledge. That's a fascinating field, and I'm sure you have a lot of exciting projects and experiences to share.\n\nWhat kind of data science work do you do, Sarvesh? Are you working in a specific industry, such as finance, healthcare, or technology? Or are you working on more general projects, like machine learning or natural language processing? I'm curious to know more about your work!", generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content="Nice to meet you, Sarvesh! As a data scientist, you must be working with data to extract insights and knowledge. That's a fascinating field, and I'm sure you have a lot of exciting projects and experiences to share.\n\nWhat kind of data science work do you do, Sarvesh? Are you working in a specific industry, such as finance, healthcare, or technology? Or are you worki

In [4]:
from langchain_core.messages import AIMessage
messages = [
    HumanMessage(content="My name is sarvesh and I am a data scientist"),
    AIMessage(content="Nice to meet you, Sarvesh. As a data scientist, you must be working with complex data sets, developing predictive models, and uncovering insights to drive business decisions. What specific areas of data science interest you the most? Are you working on any exciting projects currently?"),
    HumanMessage(content="hey what is my name and what do i do?"),
]

response = model.invoke(messages)
response

AIMessage(content="Your name is Sarvesh, and you're a data scientist.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 124, 'total_tokens': 139, 'completion_time': 0.029533715, 'completion_tokens_details': None, 'prompt_time': 0.022913776, 'prompt_tokens_details': None, 'queue_time': 0.058657244, 'total_time': 0.052447491}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019be969-e869-70e1-9d9e-d8ec49deba7f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 124, 'output_tokens': 15, 'total_tokens': 139})

### Message history
Message history in a chatbot is a stored record of the entire conversation (user inputs and AI responses) that provides context and continuity, allowing the AI to understand previous turns and generate relevant, coherent replies, rather than treating each message as a new, isolated request, often managed as a list of messages with metadata

In [5]:
from langchain_community.chat_message_histories import  ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory 

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]    

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [6]:
config = {"configurable":{"session_id":"chat1"}}

In [7]:
responce = with_message_history.invoke( 
    [HumanMessage(content="My name is sarvesh and I am a data scientist")],
    config=config
)


In [8]:
responce.content

"Nice to meet you, Sarvesh! As a data scientist, you must be working with data to extract insights and knowledge. That's a fascinating field, and I'm sure you have a lot of exciting projects and experiences to share.\n\nWhat kind of data science work do you do, Sarvesh? Are you working in a specific industry, such as finance, healthcare, or technology? Or are you working on more general projects, like machine learning or natural language processing? I'm curious to know more about your work!"

In [9]:
responce = with_message_history.invoke( 
    [HumanMessage(content="hey what is the my name ")],
    config=config
)
responce.content

"Your name is Sarvesh, and you're a data scientist."

In [10]:
### change the config --- session id
config1 = {"configurable":{"session_id":"chat2"}}

responce = with_message_history.invoke( 
    [HumanMessage(content="hey what is the my name ")],
    config=config1
)
responce.content

"I don't have any information about your name. This is the beginning of our conversation, and I don't have any prior knowledge about you. If you'd like to share your name, I'd be happy to chat with you and get to know you better! What's your name?"

### Prompt Templates

LangChain uses Prompt Templates to create dynamic and reusable prompts by incorporating variables and formatting options. They are essential for structuring interactions with language models (LLMs) and chat models. 


In [11]:
## prompt template 
from langchain_core.prompts import  ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(

[
    ( "system","You are a helpful assitant. answer all the question to the nest of your abilit"),
    MessagesPlaceholder(variable_name="message")
]
)

chain = prompt|model







In [12]:
chain.invoke({"message": [HumanMessage(content="Hii my name is sarvesh")]})

AIMessage(content="Hello Sarvesh! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to assist you with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 61, 'total_tokens': 114, 'completion_time': 0.096827284, 'completion_tokens_details': None, 'prompt_time': 0.002173601, 'prompt_tokens_details': None, 'queue_time': 0.053324789, 'total_time': 0.099000885}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019be969-ef95-7551-806f-c42e431e7865-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 61, 'output_tokens': 53, 'total_tokens': 114})

In [13]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [14]:
### change the config --- session id

config = {"configurable":{"session_id":"chat3"}}

responce = with_message_history.invoke( 
    [HumanMessage(content="hey  my name is sarvesh")],
    config=config
)
responce.content

"Hello Sarvesh! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to assist you with any questions or topics you'd like to discuss. How's your day going so far?"

In [15]:
## prompt template with add more complexity
from langchain_core.prompts import  ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(

[
    ( "system","You are a helpful assitant. answer all the question to the best of your abilit in {language}"),
    MessagesPlaceholder(variable_name="message")
]
)

chain = prompt|model


In [16]:
responce = chain.invoke({"message": [HumanMessage(content="Hii my name is sarvesh")],"language":"Hindi"}) 

In [17]:
responce

AIMessage(content='नमस्ते सर्वेश! मैं आपका सहायक हूँ। मैं आपकी किस प्रकार से मदद कर सकता हूँ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 63, 'total_tokens': 100, 'completion_time': 0.103473387, 'completion_tokens_details': None, 'prompt_time': 0.011831549, 'prompt_tokens_details': None, 'queue_time': 0.056360021, 'total_time': 0.115304936}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019be969-f2a9-7a43-8ea4-8423a98b8610-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 63, 'output_tokens': 37, 'total_tokens': 100})

######  lets how to wrap this more complicated chain in a message history class . this time , because there are multiple keys in the input , we need to specify to the correct key to use to save the chat history 

In [ ]:
## Wrap with message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="message"
    )

In [25]:
### change the config --- session id





config = {"configurable": {"session_id": "chat4"}}

response = with_message_history.invoke(
    {
        "message": HumanMessage(content="hey my name is sarvesh"),
        "language": "Hindi"
    },
    config=config
)

response.content





'नमस्ते सर्वेश! मैं आपकी मदद करने के लिए तैयार हूँ। आपका दिन कैसा गुजर रहा है? क्या आपके पास कोई प्रश्न या समस्या है जिसमें मैं आपकी सहायता कर सकता हूँ?'

##### Managing the Conversation History

One important concept to understand when building chatots is how to manage conversation history . If left unmanaged , the list of messages will grow unbounded and potentially overflow the context window of the llm , Therefore , it is important to add a step that limit the size of the message you are passing 

trim_messages = helper to reduce how many messages we are sending to the model the trimer allows us to specify how many tokens we want to keep , along with other parameter like if we want to always keep the system message and weather to allow partial message 


In [37]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

message = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hello"),
    AIMessage(content="Hi! How can I help you?"),
    HumanMessage(content="Explain LangChain in simple terms")
]

trimmer.invoke(message)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi! How can I help you?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Explain LangChain in simple terms', additional_kwargs={}, response_metadata={})]

In [38]:
## Using chains to pass one by one 

from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(message=itemgetter("message")|trimmer)
    | prompt
    | model

)


responce = chain.invoke(
    { 
    "message":message  + [HumanMessage(content="Explain LangChain in simple terms")],
    "language": "English"
    }
)

responce.content

'LangChain is a framework that helps build applications using large language models (LLMs) like AI chatbots. Here\'s a simplified explanation:\n\n**What is LangChain?**\nLangChain is an open-source framework that allows developers to create applications that interact with large language models. It provides a set of tools and libraries to make it easier to build, deploy, and manage LLM-based applications.\n\n**Key Components:**\n\n1. **Language Models**: These are the AI models that understand and generate human-like language. Examples include LLaMA, BERT, and transformer-based models.\n2. **Agents**: These are the components that interact with the language models. Agents can be thought of as "wrappers" around the language models, allowing them to perform specific tasks.\n3. **Chains**: A chain is a sequence of agents that work together to accomplish a task. Each agent in the chain processes the input and passes the output to the next agent, creating a pipeline of processing steps.\n\n*

In [40]:
## lets wrap this in the message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="message"
    )

## change the session id 
config = {"configurable": {"session_id": "chat5"}}


## invoke the model
response = with_message_history.invoke(
    {
        "message": HumanMessage(content="what are the component of langchain"),
        "language": "English"
    },
    config=config
)

response.content


"LangChain is an open-source framework for building applications that utilize large language models (LLMs). The components of LangChain can be broken down into several key parts:\n\n1. **LLM (Large Language Model)**: This is the core component of LangChain, which is a pre-trained language model that can understand and generate human-like text. Examples of LLMs include transformer-based models like BERT, RoBERTa, and XLNet.\n2. **Agent**: The agent is the interface between the user and the LLM. It receives input from the user, processes it, and then sends the input to the LLM for processing. The agent can also receive output from the LLM and post-process it before returning it to the user.\n3. **Index**: The index is a data structure that stores information about the LLM's knowledge and capabilities. It allows the agent to quickly look up information about the LLM's strengths and weaknesses, and to select the most suitable LLM for a given task.\n4. **Chain**: The chain is a sequence of 